In [57]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset
import torchvision.datasets as datasets
import torchvision.models as models
from torch.optim import lr_scheduler
import torchvision.transforms as transforms
import torchvision
from urllib.request import urlopen
import matplotlib.pyplot as plt

In [58]:
# GPU
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print('GPU State:', device)

GPU State: cuda:0


In [60]:
# Data Pre-process #
# for train data
class image_dataset(Dataset):
    def __init__(self, csvFile, rootPath, transform):
        self.df = pd.read_csv(csvFile)
        self.rootPath = rootPath
        self.xTrain = self.df['id']
        self.yTrain = pd.factorize(self.df['label'])[0]
        self.transform = transform     
    def __getitem__(self, index):
        # To read image file
        zeros = ''
        while len(str(self.df.iloc[index,0])+zeros)<6:
          zeros = zeros+'0'
        img_id = zeros+str(self.df.iloc[index,0])
        img = Image.open(os.path.join(self.rootPath,img_id+'.jpg'))
        img = img.convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        return img, self.yTrain[index]
    def __len__(self):
        return len(self.xTrain.index)
        
# for test data
class image_dataset_test(Dataset):
    def __init__(self, rootPath, transform):
        self.rootPath = rootPath
        self.fileList = os.listdir(self.rootPath)
        self.transform = transform    
    def __getitem__(self, index):
        img = Image.open(os.path.join(self.rootPath,self.fileList[index]))
        img = img.convert('RGB')
        id = int(self.fileList[index].split('.jpg')[0])
        if self.transform is not None:
            img = self.transform(img)
        return img, id
    def __len__(self):
        return len(self.fileList)


In [61]:
def test(net, testLoader, criterion):
    out = open("drive/My Drive/交大電子109上/深度學習_影像/Homework1/out.csv",'w') # open the output file to write result
    df = pd.read_csv("/content/drive/My Drive/交大電子109上/深度學習_影像/Homework1/training_labels.csv") # the csv file is same as training label
    labels = pd.factorize(df['label'])[1]
    net.eval()
    for (x, id) in testLoader:
        x = x.to(device)
        output = net(x)
        _, predicted = torch.max(output.data, 1)
        id = id.numpy()
        for i in range(len(id)):
          out.write(str(id[i])+','+labels[predicted[i]]+'\n')

In [62]:
# Training model #
def train(net, trainLoader, testLoader, optimizer, criterion, scheduler, epochs):
    testAccuracy = 0
    net.train() # Set model to training mode
    for i in range(epochs):
        totalLoss = 0
        accuracy = 0
        count = 0
        scheduler.step()
        for (x,label) in trainLoader:
            x, label = x.to(device), label.to(device)
            optimizer.zero_grad()
            output = net(x)
            _, predicted = torch.max(output.data, 1)
            loss = criterion(output, label)
            loss.backward()
            optimizer.step()
            count += len(x)
            accuracy += (predicted == label).sum().item()
            totalLoss += loss.item()*len(label)    
        print("Train Loss: {}".format(totalLoss / count))
        print("count=",count)
        print("Train Accuracy: {}".format(accuracy / count))
    return net
    

In [63]:
def main():
    # load data #
    # Data augmentation
    inputSize = 224
    dataTransformsTrain = transforms.Compose([
     transforms.RandomResizedCrop(inputSize),
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    dataTransformsValid = transforms.Compose([
     transforms.Resize(inputSize),
     transforms.CenterCrop(inputSize),
     transforms.ToTensor(),
     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    training_dataset = image_dataset(csvFile="/content/drive/My Drive/交大電子109上/深度學習_影像/Homework1/training_labels.csv",
                 rootPath="drive/My Drive/交大電子109上/深度學習_影像/Homework1/training_data/",
                transform=dataTransformsTrain)
    
    validation_dataset = image_dataset_test(rootPath="/content/drive/My Drive/交大電子109上/深度學習_影像/Homework1/testing_data/testing_data/",
                transform=dataTransformsValid)
    dataloadersTrain = torch.utils.data.DataLoader(training_dataset, batch_size=32, shuffle=True)
    dataloadersValid = torch.utils.data.DataLoader(validation_dataset, batch_size=32, shuffle=False)
    
    # model (transfer learning) #
    num_classes = 196
    model = models.resnet18(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
    model = model.to(device)

    # set optimization function #
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.0001)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.3)
    # set loss function #
    criterion = nn.CrossEntropyLoss().to(device)
        
    # training #
    model_ft = train(model, dataloadersTrain, dataloadersValid, optimizer, criterion, scheduler, 60)
    
    # Output test result #
    test(model_ft, dataloadersValid, criterion)

In [64]:
main()

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Train Loss: 4.630559571257222
count= 11185
Train Accuracy: 0.10067054090299508
Train Loss: 3.3265776876578332
count= 11185
Train Accuracy: 0.3466249441215914
Train Loss: 2.4732000500376756
count= 11185
Train Accuracy: 0.518998658918194
Train Loss: 1.9109434769310267
count= 11185
Train Accuracy: 0.6226195797943674
Train Loss: 1.5545388085958305
count= 11185
Train Accuracy: 0.6813589628967367
Train Loss: 1.3166145963332032
count= 11185
Train Accuracy: 0.7238265534197587
Train Loss: 1.1578772161920314
count= 11185
Train Accuracy: 0.751721054984354
Train Loss: 1.0162732550647038
count= 11185
Train Accuracy: 0.7800625838176128
Train Loss: 0.9205984902264659
count= 11185
Train Accuracy: 0.7925793473401878
Train Loss: 0.8593758487307072
count= 11185
Train Accuracy: 0.8103710326329906
Train Loss: 0.7899937283102828
count= 11185
Train Accuracy: 0.8253911488600805
Train Loss: 0.7552374784041581
count= 11185
Train Accuracy: 0.8279839070183281
Train Loss: 0.7013630645505328
count= 11185
Train Accu